In [34]:
# Import dependencies

import numpy as np
import rasterio
import os

# Functions

# 1. Clip the Sentinel-1 bands to the same extent as the Sentinel-2 imagery 

In [8]:
labelpath ='/mnt/d/SabineMSI/s2classifications'
man_ims = [os.path.join(labelpath, file) for file in os.listdir(os.path.join(labelpath, f'manual')) if file.endswith('.tif')]

In [32]:
# set the directory for where your images are located
labelpath ='/mnt/d/SabineMSI/s2classifications'
man_ims = [os.path.join(labelpath, f'manual/{file}') for file in os.listdir(os.path.join(labelpath, f'manual')) if file.endswith('.tif')]
otsu_ims = [os.path.join(labelpath, f'otsu/{file}') for file in os.listdir(os.path.join(labelpath, f'otsu')) if file.endswith('.tif')]
kmeans_ims = [os.path.join(labelpath, f'kmeans/{file}') for file in os.listdir(os.path.join(labelpath, f'kmeans')) if file.endswith('.tif')]
gmm_ims = [os.path.join(labelpath, f'gmm{file}') for file in os.listdir(os.path.join(labelpath, f'gmm')) if file.endswith('.tif')]

grdpath = '/mnt/d/SabineGRD/GEE'
orig_ims = [os.path.join(grdpath, f'Original/{file}') for file in os.listdir(os.path.join(grdpath, f'Original')) if file.endswith('.tif')]
filt_ims = [os.path.join(grdpath, f'GammaMap{file}') for file in os.listdir(os.path.join(grdpath, f'GammaMap')) if file.endswith('.tif')]

In [17]:
# Load the VV or VH band from a Sentinel-1 image_vv
with rasterio.open(orig_ims[90]) as src:
    image_vv = src.read(1)
    image_vh = src.read(2)

138

# 2. Register Sentinel-1 Imagery to Sentinel-2 labeled data

# 3. Label the Sentinel-1 data as land or water

# 4. Save labeled S1 imagery for training Neural Network